In [12]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

df = pd.read_csv('../../data/ST001937_AN003150.csv')

In [13]:
df = df.drop(['Sample ID', 'RAW_FILE_NAME'], axis=1)
class_counts = df.groupby('Phenotypes').size()
print(class_counts)
df['Phenotypes'] = df['Phenotypes'].apply(lambda x: 0 if x == 'Healthy Controls' else 1 if x == 'Maligant SPNS' else 2)

Phenotypes
Benign SPNS         157
Healthy Controls    280
Maligant SPNS       723
dtype: int64


In [14]:
#performing orthogonal pls-da on the data set
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

X = df.drop('Phenotypes', axis=1)
y = df['Phenotypes']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pls = PLSRegression(n_components=2)
pls.fit(X_train, y_train)
y_pred = pls.predict(X_test)
y_pred = y_pred.argmax(axis=1)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


0.21551724137931033
              precision    recall  f1-score   support

           0       0.22      1.00      0.35        50
           1       0.00      0.00      0.00       151
           2       0.00      0.00      0.00        31

    accuracy                           0.22       232
   macro avg       0.07      0.33      0.12       232
weighted avg       0.05      0.22      0.08       232

[[ 50   0   0]
 [151   0   0]
 [ 31   0   0]]


/home/marco/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/marco/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/marco/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion matrix